# Music Composer Identification using Deep Learning

The primary objective of this project is to develop a deep learning model that can predict the composer of a given musical score accurately. The project aims to accomplish this objective by using two deep learning techniques: Long Short-Term Memory (LSTM) and Convolutional Neural Network (CNN).

## Project Team & Responsibilities:

* **Dom:** Data Collection, Data Preprocessing (MIDI conversion, segmentation, augmentation), Feature Extraction (Piano Rolls for CNN, Sequential Features for LSTM).
* **Santosh:** CNN Model Building, Training, Evaluation, Optimization.
* **Jim:** LSTM Model Building, Training, Evaluation, Optimization.

## Project Roadmap & Status:

Here's a breakdown of our project phases and current status:

1.  **Initial Setup & Data Download (COMPLETED by Jim):**
    * Basic imports are set up.
    * The `blanderbuss/midi-classic-music` dataset has been downloaded from Kaggle.
    * *Status:* Ready for data processing.

2.  **Data Preprocessing & Feature Extraction (COMPLETED by Dom):**
    * **Goal:** Convert raw MIDI files into numerical features (Piano Rolls for CNNs, Sequential Features for LSTMs) and augment dataset.
    * **Responsible:** Dom.
    * *Current Status:* Completed / Needs implementation of the sections below.

3.  **Model Building (NEXT STEP for Team):**
    * **Goal:** Design CNN and LSTM model architectures.
    * **Responsible:** Santosh (CNN), Jim (LSTM).
    * *Dependencies:* Requires processed data from Phase 2.

4.  **Model Training & Evaluation (AFTER Model Building):**
    * **Goal:** Train the models and evaluate their performance using metrics like accuracy, precision, and recall.
    * **Responsible:** Santosh (CNN), Jim (LSTM).
    * *Dependencies:* Requires built models from Phase 3.

5.  **Model Optimization (Post Training):**
    * **Goal:** Fine-tune model hyperparameters to improve performance.
    * **Responsible:** Santosh (CNN), Jim (LSTM) & Dom (Feature Engineering).
    * *Dependencies:* Requires initial model training.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

Data Collection
The dataset contains the midi files of compositions from well-known classical composers like Bach, Beethoven, Chopin, and Mozart. The dataset has been labeled with the name of the composer for each score. Predictions are performed for only the below composers:

1-Bach

2-Beethoven

3-Chopin

4-Mozart

In [2]:
%pip install kagglehub

import kagglehub

# Download latest version
path = kagglehub.dataset_download("blanderbuss/midi-classic-music")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/midi-classic-music


In [3]:
import os

# List all files in the dataset path
for root, dirs, files in os.walk(path):
    for file in files:
        #print(os.path.join(root, file))
        # Check if the file is a MIDI file and contains 'bach' in its name.
        # There are other composers that need to be processed too.
        if (file.endswith('.mid') or file.endswith('.midi')) and 'bach' in file.lower():
            print(f"Found MIDI file: {file}")
            # Add file to Bach dataset processing logic here


Found MIDI file: Liszt Bach Prelude Transcription.mid
Found MIDI file: C.P.E.Bach Solfeggieto.mid
Found MIDI file: Piano version of Bachs two part inventions No.15.mid
Found MIDI file: Piano version of Bachs two part inventions No.6.mid
Found MIDI file: Piano version of Bachs two part inventions No.7.mid
Found MIDI file: Piano version of Bachs two part inventions No.4.mid
Found MIDI file: Piano version of Bachs two part inventions No...mid
Found MIDI file: Piano version of Bachs two part inventions No.1.mid
Found MIDI file: Piano version of Bachs two part inventions No.9.mid
Found MIDI file: Piano version of Bachs two part inventions No.14.mid
Found MIDI file: Piano version of Bachs two part inventions No.10.mid
Found MIDI file: Piano version of Bachs two part inventions No.13.mid
Found MIDI file: Piano version of Bachs two part inventions No.12.mid
Found MIDI file: Piano version of Bachs two part inventions No.11.mid
Found MIDI file: Piano version of Bachs two part inventions No.5.mid

Convert MIDI file to something useful for LSTM and CNN.

In [4]:
# Imports
import os
import glob
import music21
import pretty_midi
import numpy as np # Already imported, but good to have here for clarity for my feature engineering
import pickle
import collections

In [5]:
# I will place these here so they run after Kaggle download, as I encountered conflicts with the initial setup when adding above.
!pip install music21
!pip install pretty_midi
!pip install --upgrade numpy # Ensure I have a recent numpy version

Data Pre-processing: Convert the musical scores into a format suitable for deep learning models. This involves converting the musical scores into MIDI files and applying data augmentation techniques.


In [6]:
# Data Preprocessing and Feature Extraction
KAGGLE_DOWNLOAD_PATH = "/root/.cache/kagglehub/datasets/blanderbuss/midi-classic-music/versions/1"
MIDI_DIR = KAGGLE_DOWNLOAD_PATH

OUTPUT_DIR = "/content/processed_data"
SEGMENT_DURATION_SECONDS = 5
SAMPLES_PER_SECOND = 100

PITCH_LOW = 21
PITCH_HIGH = 108
NUM_PITCHES = PITCH_HIGH - PITCH_LOW + 1

AUGMENT_TRANSPOSITION_STEPS = [-3, -2, -1, 1, 2, 3]
AUGMENT_TEMPO_SCALES = [0.9, 1.1]

# Defines composers
COMPOSERS = ["Bach", "Beethoven", "Chopin", "Mozart"]

# Creates output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"MIDI data will be processed from: {MIDI_DIR}")
print(f"Processed data will be saved to: {OUTPUT_DIR}")

MIDI data will be processed from: /root/.cache/kagglehub/datasets/blanderbuss/midi-classic-music/versions/1
Processed data will be saved to: /content/processed_data


### 2.2 Feature Extraction : Extract features from the MIDI files, such as notes, chords, and tempo, using music analysis tools.

Here, the prepared MIDI segments are converted into numerical representations. I've generated different formats for the CNN and LSTM models to leverage the strengths of each.

* **For CNNs: The Piano Roll**
    * **Purpose:** CNNs excel at recognizing visual patterns. A piano roll converts music into a 2D image (pitch vs. time), allowing the CNN to "see" and learn characteristic melodic shapes, harmonic voicings, and rhythmic patterns that define a composer's style.
    * **Details:** The piano roll captures note activity (velocity) across a defined pitch range (MIDI 21-108) over time, sampled at 100 samples per second. All outputs are normalized to [0,1] and padded/truncated to a consistent shape.
* **For LSTMs: Sequential Features (Chroma & Note Density)**
    * **Purpose:** LSTMs are great tools for understanding temporal sequences. These features describe the harmonic content and musical activity at each point in time, allowing the LSTM to learn how a composer's musical ideas evolve.
    * **Details:** Each time step in the sequence contains a 12-element Pitch Class Profile (Chroma) representing harmonic presence (e.g., C, C#, D) and a single value for overall note density/volume. These are also sampled at 100 samples per second and normalized.

In [7]:
# Feature Extraction - midi_to_sequential_features (for LSTMs)
# This function extracts time-series features like Pitch Class Profiles and note density from a MIDI segment for LSTMs

def midi_to_sequential_features(midi_data_segment: pretty_midi.PrettyMIDI, duration: float,
                                samples_per_second: int, pitch_low: int, pitch_high: int) -> np.ndarray:
    if not midi_data_segment.instruments:
        return None

    num_target_time_steps = int(duration * samples_per_second)
    num_features_per_timestep = 12 + 1 # Chroma + Note Density
    sequential_features = np.zeros((num_target_time_steps, num_features_per_timestep), dtype=np.float32)

    chroma_features = midi_data_segment.get_chroma(fs=samples_per_second)
    chroma_features = chroma_features.T

    note_density = np.zeros(num_target_time_steps, dtype=np.float32)
    for instrument in midi_data_segment.instruments:
        for note in instrument.notes:
            start_idx = int(note.start * samples_per_second)
            end_idx = int(note.end * samples_per_second)
            start_idx = max(0, min(start_idx, num_target_time_steps - 1))
            end_idx = max(0, min(end_idx, num_target_time_steps - 1))
            if end_idx >= start_idx:
                note_density[start_idx:end_idx] += note.velocity

    max_density = np.max(note_density)
    if max_density > 0:
        note_density /= max_density

    sequential_features[:, :12] = chroma_features[:num_target_time_steps, :]
    sequential_features[:, 12] = note_density[:num_target_time_steps]

    current_time_steps = sequential_features.shape[0]
    if current_time_steps < num_target_time_steps:
        padding_needed = num_target_time_steps - current_time_steps
        sequential_features = np.pad(sequential_features, ((0, padding_needed), (0, 0)), mode='constant')
    elif current_time_steps > num_target_time_steps:
        sequential_features = sequential_features[:num_target_time_steps, :]
    return sequential_features

In [8]:
# Feature Extraction - midi_to_piano_roll (for CNNs)
# This function converts a MIDI segment into a 2D image-like "piano roll" for CNNs.

def midi_to_piano_roll(midi_data_segment: pretty_midi.PrettyMIDI, duration: float,
                        samples_per_second: int, pitch_low: int, pitch_high: int) -> np.ndarray:
    if not midi_data_segment.instruments:
        return None

    piano_roll = midi_data_segment.get_piano_roll(fs=samples_per_second,
                                                low=pitch_low, high=pitch_high)
    piano_roll = piano_roll / 127.0

    num_target_time_steps = int(duration * samples_per_second)
    current_time_steps = piano_roll.shape[1]

    if current_time_steps < num_target_time_steps:
        padding_needed = num_target_time_steps - current_time_steps
        piano_roll = np.pad(piano_roll, ((0, 0), (0, padding_needed)), mode='constant')
    elif current_time_steps > num_target_time_steps:
        piano_roll = piano_roll[:, :num_target_time_steps]

    return piano_roll.reshape(NUM_PITCHES, num_target_time_steps, 1)

In [9]:
# Utility Function - create_pretty_midi_segment
# This function extracts a specific time segment from a larger MIDI file.

def create_pretty_midi_segment(full_midi_data: pretty_midi.PrettyMIDI, start_time: float, end_time: float) -> pretty_midi.PrettyMIDI:
    segment_pm = pretty_midi.PrettyMIDI()
    for instrument in full_midi_data.instruments:
        new_instrument = pretty_midi.Instrument(program=instrument.program, is_drum=instrument.is_drum, name=instrument.name)
        for note in instrument.notes:
            if note.end > start_time and note.start < end_time:
                new_note = pretty_midi.Note(
                    velocity=note.velocity,
                    pitch=note.pitch,
                    start=max(0.0, note.start - start_time),
                    end=min(end_time - start_time, note.end - start_time)
                )
                if new_note.end > new_note.start:
                    new_instrument.notes.append(new_note)
        if new_instrument.notes:
            segment_pm.instruments.append(new_instrument)
    return segment_pm

In [10]:
# Utility Function - apply_augmentation
# This function modifies a MIDI segment by transposing its pitch or scaling its tempo.

def apply_augmentation(midi_data_segment: pretty_midi.PrettyMIDI, augmentation_type: str, value) -> pretty_midi.PrettyMIDI:
    augmented_midi = pretty_midi.PrettyMIDI()
    for instrument in midi_data_segment.instruments:
        new_instrument = pretty_midi.Instrument(program=instrument.program, is_drum=instrument.is_drum, name=instrument.name)
        for note in instrument.notes:
            new_note = pretty_midi.Note(note.velocity, note.pitch, note.start, note.end)
            new_instrument.notes.append(new_note)
        augmented_midi.instruments.append(new_instrument)

    if augmentation_type == 'transpose':
        for instrument in augmented_midi.instruments:
            for note in instrument.notes:
                note.pitch = int(max(0, min(127, note.pitch + value)))
    elif augmentation_type == 'tempo_scale':
        for instrument in augmented_midi.instruments:
            for note in instrument.notes:
                note.start *= value
                note.end *= value
    else:
        raise ValueError(f"Unknown augmentation type: {augmentation_type}")
    return augmented_midi

### Data Processing Loop & Output Conclusion

This section orchestrates the loading of MIDI files, segmenting them, applying all augmentations, extracting features, and finally saving the processed data.

* **Process:** Iterates through each composer's MIDI files, segments them, applies both transposition and tempo scaling for each segment, and then generates both CNN and LSTM features.
* **Output Data:** The processed features and corresponding labels are saved as `.pkl` files in the `/content/processed_data/` directory.

---

#### **The data is ready for model training!**

* **For CNN Model (Santosh):**
    * Load `features_cnn.pkl`.
    * Expected input shape: `(num_segments, 88, 500, 1)` - (total samples, pitches, time steps, channels).
* **For LSTM Model (Jim):**
    * Load `features_lstm.pkl`.
    * Expected input shape: `(num_segments, 500, 13)` - (total samples, time steps, features per time step).
* **Labels:**
    * Load `labels.pkl` (numerical labels corresponding to composers).
    * Load `composer_to_label.pkl` and `label_to_composer.pkl` to map between numerical labels and composer names.

YOu can/should convert these NumPy arrays to PyTorch tensors for your models (e.g., `torch.tensor(data, dtype=torch.float32)` for features, `torch.tensor(labels, dtype=torch.long)` for labels).


Model Building: Develop a deep learning model using LSTM and CNN architectures to classify the musical scores according to the composer.


Model Training: Train the deep learning model using the pre-processed and feature-extracted data.


Model Evaluation: Evaluate the performance of the deep learning model using accuracy, precision, and recall metrics.


Model Optimization: Optimize the deep learning model by fine-tuning hyperparameters.